In [1]:
import numpy as np 
import pickle

In [8]:
!pwd

/n/home12/binxuwang/Github/DiffusionReasoning/notebooks


In [12]:
data_dir = '/n/home12/binxuwang/Github/DiffusionReasoning/rule_data/'

# attr_all = np.load(data_dir+'attr_all.npy') # (40, 12000, 3, 9, 3), (num_rules, num_samples, num_panels, num_pos, num_attributes)

with open(data_dir+'r_dict_M7_withR.pkl', 'rb') as file:
    r_dict_M7_withR = pickle.load(file)
    
with open(data_dir+'r_dict_M10_withR.pkl', 'rb') as file:
    r_dict_M10_withR = pickle.load(file)
    
with open(data_dir+'r_dict_num_withR.pkl', 'rb') as file:
    r_dict_num_withR = pickle.load(file)
    
with open(data_dir+'r_dict_pos_withR.pkl', 'rb') as file:
    r_dict_pos_withR = pickle.load(file)

In [13]:
def get_key_first2(a_list):
    """
    convert a_list to key string
    e.g., a_list = [[1,2], [1]], key='12-1'
    """
    key = ''
    for x in a_list[0]: 
        key+=str(x)
    key += '-'
    for x in a_list[1]: 
        key+=str(x) 
    return key

def get_attr(attr): 
    """attr.shape # (num_panels, num_obs, obj)"""
    # step 1: check number of objects 
    x_num = [[] for i in range(3)]
    x_pos = [[] for i in range(3)]
    x_shape = [[] for i in range(3)]
    x_color = [[] for i in range(3)]
    x_size = [[] for i in range(3)]

    for i, attr_panel in enumerate(attr): 

        pos = list(np.where((attr_panel==-1).sum(1)==0)[0])
        x_pos[i] = pos 
        x_num[i] = [len(pos)]
        x_shape[i] = list(np.unique(attr_panel[pos,0]))
        x_color[i] = list(np.unique(attr_panel[pos,1]))
        x_size[i] = list(np.unique(attr_panel[pos,2]))
        
    return x_num, x_pos, x_shape, x_color, x_size 

def check_rule(x, r_dict_withR): 
    i_R = None
    key_first2 = get_key_first2(x) # '5-5'
    if key_first2 in r_dict_withR.keys(): 
        d = r_dict_withR[key_first2]
        if x[2] in d.values():
            i_R = list(d.keys())[list(d.values()).index(x[2])]
    return i_R 

def check_row_rule(attr): 
    """attr: (3, 9, 3), return list of rule """
    x_num, x_pos, x_shape, x_color, x_size = get_attr(attr)
    
    rule_list = []
    R_shape = check_rule(x_shape, r_dict_M7_withR)
    if R_shape is not None: 
        rule_list.append(R_shape)

    R_color = check_rule(x_color, r_dict_M10_withR)
    if R_color is not None: 
        rule_list.append(R_color+10)

    R_size = check_rule(x_size, r_dict_M10_withR)
    if R_size is not None: 
        rule_list.append(R_size+20)
        
    R_num = check_rule(x_num, r_dict_num_withR)
    if R_num is not None: 
        rule_list.append(R_num+30)
        
    R_pos = check_rule(x_pos, r_dict_pos_withR)
    if R_pos is not None: 
        rule_list.append(R_pos+37)
        
    return np.asarray(rule_list)

In [14]:
import torch
import einops
train_attrs = torch.load('/n/home12/binxuwang/Github/DiffusionReasoning/train_inputs.pt')
train_row_img = einops.rearrange(train_attrs, 'c s pnl (H W) attr -> c s attr H (pnl W)', H=3, W=3, attr=3, pnl=3)
train_row_img.shape

torch.Size([35, 10000, 3, 3, 9])

In [15]:
train_attrs.shape

torch.Size([35, 10000, 3, 9, 3])

In [16]:
for i_rule in range(35): 
    attr = train_attrs[i_rule, 0]
    rule_list = check_row_rule(attr)
    print(rule_list)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[31]
[32]
[33]
[34]
[35]
[36]
[38]
[39]
